### Importação e Download das bibliotecas

###### Importa bibliotecas necessárias para o processamento de texto, modelagem, validação, e manipulação de dados. Incluindo o download de pacotes de stopwords e tokenização do NLTK.

In [1]:
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd
import warnings
from sklearn.exceptions import UndefinedMetricWarning

# Baixar stopwords em português
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')
nltk.download('punkt_tab')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping stemmers\rslp.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


### Remoção de warnings
###### Configura o Python para ignorar certos tipos de avisos para manter a saída limpa.

In [2]:
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=UserWarning)

### Processamento dos dados


###### Realiza a limpeza e transformação do texto, convertendo-o para um formato utilizável no treinamento do modelo de machine learning, incluindo a vetorização e binarização das tags.

In [3]:
# Função para pré-processar o texto: converte para minúsculas, remove pontuações, tokeniza, remove stopwords e aplica stemming.
def preprocess_text(text):
    # Converte para minúsculas
    text = text.lower()
    # Remove pontuações e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    # Tokeniza o texto
    tokens = word_tokenize(text)
    # Remove stopwords em português
    tokens = [word for word in tokens if word not in stopwords.words('portuguese')]
    # Junta os tokens de volta em um texto processado
    return ' '.join(tokens)

In [4]:
csv_file = '../../../data/processed/processed_data.csv'

df = pd.read_csv(csv_file)

# Remover linhas onde texto ou tags estejam ausentes
df = df.dropna(subset=['text', 'tags'])

In [5]:
df['processed_text'] = df['text'].apply(preprocess_text)

In [6]:
vectorizer = TfidfVectorizer(max_features=50000)
X = vectorizer.fit_transform(df['processed_text'])

In [7]:
# Convertendo as tags para um formato binário que pode ser usado no treinamento
y = df['tags'].str.get_dummies(sep=', ')

### Treino e Teste do Modelo

###### Divide o conjunto de dados em treino e teste e treina um modelo de regressão logística multiclasse.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [9]:
model = OneVsRestClassifier(LogisticRegression())
model.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [10]:
y_pred = model.predict(X_test)

### Validação e Reports do Modelo
###### Gera o relatório de classificação para avaliar a performance do modelo com base no conjunto de teste.

In [11]:
report = classification_report(y_test, y_pred, target_names=y.columns)

In [12]:
print(report)

                                                precision    recall  f1-score   support

                  Administradores de Carteiras       1.00      1.00      1.00        15
 Agencias de Classificação de Risco de Crédito       1.00      1.00      1.00        15
              Analistas de Valores Mobiliários       1.00      1.00      1.00        15
                    Assessores de Investimento       0.93      1.00      0.97        14
                          Auditor Independente       0.00      0.00      0.00         1
           Cadastro de Participantes Regulados       0.73      1.00      0.85        11
                        Clubes de Investimento       0.87      1.00      0.93        13
                                     Companhia       0.00      0.00      0.00         0
                         Comunicado ao Mercado       0.00      0.00      0.00         0
            Consultores de Valores Mobiliários       1.00      1.00      1.00        15
                          Decis

In [13]:
# Realiza a validação cruzada para avaliar a robustez do modelo em diferentes subconjuntos de dados.
scores = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'f1_weighted'], return_train_score=False)

# Resultados da validação cruzada
print("Accuracy Scores:", scores['test_accuracy'])
print("F1 Scores:", scores['test_f1_weighted'])
print("Média de Accuracy:", scores['test_accuracy'].mean())
print("Média de F1:", scores['test_f1_weighted'].mean())

Accuracy Scores: [0.06666667 0.26666667 0.46666667 0.         0.14285714]
F1 Scores: [0.8612805  0.92041024 0.92898385 0.88024364 0.87102646]
Média de Accuracy: 0.18857142857142856
Média de F1: 0.8923889354384438


### Exportação do Modelo

###### Salva o modelo treinado, o vetorizador TF-IDF, e o binarizador de tags em arquivos para reutilização futura.

In [14]:
# Salvar o modelo
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Salvar o vetor TF-IDF
with open('vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

# Salvar o binarizador de tags
with open('mlb.pkl', 'wb') as mlb_file:
    pickle.dump(y.columns, mlb_file)